In [121]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

import optuna
import torch


In [122]:
df = pd.read_csv('insurance.csv')
df

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category
0,67,119.8,1.56,2.92000,False,Jaipur,retired,High
1,36,101.1,1.83,34.28000,False,Chennai,freelancer,Low
2,39,56.8,1.64,36.64000,False,Indore,freelancer,Low
3,22,109.4,1.55,3.34000,True,Mumbai,student,Medium
4,69,62.2,1.60,3.94000,True,Indore,retired,High
...,...,...,...,...,...,...,...,...
95,36,52.8,1.57,19.64000,False,Indore,business_owner,Low
96,26,113.8,1.54,34.01000,False,Delhi,private_job,Low
97,52,60.8,1.80,44.86000,False,Hyderabad,freelancer,Low
98,27,101.1,1.82,28.30000,False,Kolkata,business_owner,Low


In [123]:
df.isnull().sum()

age                           0
weight                        0
height                        0
income_lpa                    0
smoker                        0
city                          0
occupation                    0
insurance_premium_category    0
dtype: int64

In [124]:
df.describe()

,age,weight,height,income_lpa
count,100.000000,100.000000,100.000000,100.000000
mean,47.180000,83.894000,1.713200,18.400600
std,16.649312,21.020278,0.110205,16.067465
min,18.000000,51.100000,1.500000,0.530000
25%,34.750000,63.650000,1.610000,2.897500
50%,47.000000,82.300000,1.730000,14.122583
75%,61.000000,101.300000,1.810000,30.162500
max,75.000000,119.800000,1.900000,50.000000


In [125]:
df_feat = df.copy()
df_feat

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category
0,67,119.8,1.56,2.92000,False,Jaipur,retired,High
1,36,101.1,1.83,34.28000,False,Chennai,freelancer,Low
2,39,56.8,1.64,36.64000,False,Indore,freelancer,Low
3,22,109.4,1.55,3.34000,True,Mumbai,student,Medium
4,69,62.2,1.60,3.94000,True,Indore,retired,High
...,...,...,...,...,...,...,...,...
95,36,52.8,1.57,19.64000,False,Indore,business_owner,Low
96,26,113.8,1.54,34.01000,False,Delhi,private_job,Low
97,52,60.8,1.80,44.86000,False,Hyderabad,freelancer,Low
98,27,101.1,1.82,28.30000,False,Kolkata,business_owner,Low


**Feature Engineering**

In [126]:
# Feature 1: BMI 
df_feat['BMI'] = df_feat['weight'] / (df_feat['height'] ** 2)
df_feat.head()

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category,BMI
0,67,119.8,1.56,2.92,False,Jaipur,retired,High,49.227482
1,36,101.1,1.83,34.28,False,Chennai,freelancer,Low,30.189017
2,39,56.8,1.64,36.64,False,Indore,freelancer,Low,21.118382
3,22,109.4,1.55,3.34,True,Mumbai,student,Medium,45.535900
4,69,62.2,1.60,3.94,True,Indore,retired,High,24.296875


In [127]:
# Feature 2: Age Group 
def age_group(age):
    if age <= 18:
        return "Child"
    elif age > 18 and age <= 40:
        return "Young Adult"
    elif age > 40 and age <= 60:
        return "Middle Aged Adult"
    else:
        return "Senior Citizen"
    

df_feat['Age Group'] = df_feat['age'].apply(age_group)
df_feat.head()

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category,BMI,Age Group
0,67,119.8,1.56,2.92,False,Jaipur,retired,High,49.227482,Senior Citizen
1,36,101.1,1.83,34.28,False,Chennai,freelancer,Low,30.189017,Young Adult
2,39,56.8,1.64,36.64,False,Indore,freelancer,Low,21.118382,Young Adult
3,22,109.4,1.55,3.34,True,Mumbai,student,Medium,45.535900,Young Adult
4,69,62.2,1.60,3.94,True,Indore,retired,High,24.296875,Senior Citizen


In [128]:
# Feature 3: Lifestyle Risk - Combination of Smoker and BMI
def lifestyle_risk(row):
    if row['smoker'] and row['BMI'] > 30:
        return "High"
    elif row['smoker'] and row['BMI'] > 25:
        return "Medium"
    else:
        return "Low"


df_feat["Lifestyle Risk"] = df_feat.apply(lifestyle_risk, axis=1)
df_feat.head()

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category,BMI,Age Group,Lifestyle Risk
0,67,119.8,1.56,2.92,False,Jaipur,retired,High,49.227482,Senior Citizen,Low
1,36,101.1,1.83,34.28,False,Chennai,freelancer,Low,30.189017,Young Adult,Low
2,39,56.8,1.64,36.64,False,Indore,freelancer,Low,21.118382,Young Adult,Low
3,22,109.4,1.55,3.34,True,Mumbai,student,Medium,45.535900,Young Adult,High
4,69,62.2,1.60,3.94,True,Indore,retired,High,24.296875,Senior Citizen,Low


In [129]:
# Feature 4: City Tier
tier_1_cities = ["Mumbai", "Delhi", "Bangalore", "Chennai", "Kolkata", "Hyderabad", "Pune"]
tier_2_cities = [
    "Jaipur", "Chandigarh", "Indore", "Lucknow", "Patna", "Ranchi", "Visakhapatnam", "Coimbatore",
    "Bhopal", "Nagpur", "Vadodara", "Surat", "Rajkot", "Jodhpur", "Raipur", "Amritsar", "Varanasi",
    "Agra", "Dehradun", "Mysore", "Jabalpur", "Guwahati", "Thiruvananthapuram", "Ludhiana", "Nashik",
    "Allahabad", "Udaipur", "Aurangabad", "Hubli", "Belgaum", "Salem", "Vijayawada", "Tiruchirappalli",
    "Bhavnagar", "Gwalior", "Dhanbad", "Bareilly", "Aligarh", "Gaya", "Kozhikode", "Warangal",
    "Kolhapur", "Bilaspur", "Jalandhar", "Noida", "Guntur", "Asansol", "Siliguri"]

def city_tier(city):
    if city in tier_1_cities:
        return 1
    elif city in tier_2_cities:
        return 2
    else:
        return 3


df_feat["City Tier"] = df_feat["city"].apply(city_tier)
df_feat.head()

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category,BMI,Age Group,Lifestyle Risk,City Tier
0,67,119.8,1.56,2.92,False,Jaipur,retired,High,49.227482,Senior Citizen,Low,2
1,36,101.1,1.83,34.28,False,Chennai,freelancer,Low,30.189017,Young Adult,Low,1
2,39,56.8,1.64,36.64,False,Indore,freelancer,Low,21.118382,Young Adult,Low,2
3,22,109.4,1.55,3.34,True,Mumbai,student,Medium,45.535900,Young Adult,High,1
4,69,62.2,1.60,3.94,True,Indore,retired,High,24.296875,Senior Citizen,Low,2


In [130]:
df_feat = df_feat[['age', 'income_lpa', 'BMI', 'smoker', 
                    'occupation', 'Age Group', 
                    'City Tier', 'Lifestyle Risk', 
                    'insurance_premium_category']]

df_feat.head()

,age,income_lpa,BMI,smoker,occupation,Age Group,City Tier,Lifestyle Risk,insurance_premium_category
0,67,2.92,49.227482,False,retired,Senior Citizen,2,Low,High
1,36,34.28,30.189017,False,freelancer,Young Adult,1,Low,Low
2,39,36.64,21.118382,False,freelancer,Young Adult,2,Low,Low
3,22,3.34,45.535900,True,student,Young Adult,1,High,Medium
4,69,3.94,24.296875,True,retired,Senior Citizen,2,Low,High


In [131]:
X = df_feat.drop(columns=['insurance_premium_category'])
y = df_feat['insurance_premium_category']

print(X.head())
print(y.head())

   age  income_lpa        BMI  ...       Age Group City Tier Lifestyle Risk
0   67        2.92  49.227482  ...  Senior Citizen         2            Low
1   36       34.28  30.189017  ...     Young Adult         1            Low
2   39       36.64  21.118382  ...     Young Adult         2            Low
3   22        3.34  45.535900  ...     Young Adult         1           High
4   69        3.94  24.296875  ...  Senior Citizen         2            Low

[5 rows x 8 columns]
0      High
1       Low
2       Low
3    Medium
4      High
Name: insurance_premium_category, dtype: object


In [132]:
df_feat.head()

,age,income_lpa,BMI,smoker,occupation,Age Group,City Tier,Lifestyle Risk,insurance_premium_category
0,67,2.92,49.227482,False,retired,Senior Citizen,2,Low,High
1,36,34.28,30.189017,False,freelancer,Young Adult,1,Low,Low
2,39,36.64,21.118382,False,freelancer,Young Adult,2,Low,Low
3,22,3.34,45.535900,True,student,Young Adult,1,High,Medium
4,69,3.94,24.296875,True,retired,Senior Citizen,2,Low,High


In [133]:
# Define categorical and numerical features

categorical_features = ['occupation', 'Age Group', 'Lifestyle Risk', 'City Tier', 'smoker']
numerical_features = ["income_lpa", 'BMI', 'age']

In [134]:
# Create a ColumnTransformer to handle categorical and numerical features

preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(), categorical_features),
    ('num', StandardScaler(), numerical_features)
])

In [135]:
# Create a pipeline with preprocessor and RandomForestClassifier

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [136]:
# Spliting the data and training the model

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['occupation', 'Age Group',
                                                   'Lifestyle Risk',
                                                   'City Tier', 'smoker']),
                                                 ('num', StandardScaler(),
                                                  ['income_lpa', 'BMI',
                                                   'age'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [137]:
# Prediction and evaluation
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.75

**Hyper Parameter Tuning using Optuna**

In [138]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 150, 500, step=10)  # Number of trees in the forest
    max_depth = trial.suggest_int('max_depth', 5, 10)  # Maximum depth of the trees
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)  # Minimum number of samples required to split an internal node
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5)  # Minimum number of samples required to be at a leaf node
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2']) # Number of features to consider when looking for the best split

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42,
        n_jobs=-1 # Use all available CPU cores for training
    )

    pipeline_optuna = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    score = cross_val_score(pipeline_optuna, X_train, y_train, cv=5, scoring='accuracy').mean()
    return score

In [139]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True)


[I 2025-12-05 18:33:33,981] A new study created in memory with name: no-name-21390e4b-b781-42a5-910d-742549deb45c
Best trial: 0. Best value: 0.725:   2%|▏         | 1/50 [00:04<03:57,  4.85s/it]

[I 2025-12-05 18:33:38,852] Trial 0 finished with value: 0.725 and parameters: {'n_estimators': 480, 'max_depth': 9, 'min_samples_split': 6, 'min_samples_leaf': 5, 'max_features': 'log2'}. Best is trial 0 with value: 0.725.


Best trial: 1. Best value: 0.7625:   4%|▍         | 2/50 [00:09<03:45,  4.69s/it]

[I 2025-12-05 18:33:43,434] Trial 1 finished with value: 0.7625 and parameters: {'n_estimators': 470, 'max_depth': 6, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_features': 'log2'}. Best is trial 1 with value: 0.7625.


Best trial: 1. Best value: 0.7625:   6%|▌         | 3/50 [00:13<03:21,  4.29s/it]

...

[I 2025-12-05 18:36:44,957] Trial 47 finished with value: 0.725 and parameters: {'n_estimators': 290, 'max_depth': 6, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 17 with value: 0.7875.


Best trial: 17. Best value: 0.7875:  98%|█████████▊| 49/50 [03:14<00:03,  3.44s/it]

[I 2025-12-05 18:36:48,995] Trial 48 finished with value: 0.7625 and parameters: {'n_estimators': 410, 'max_depth': 7, 'min_samples_split': 9, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 17 with value: 0.7875.


Best trial: 17. Best value: 0.7875: 100%|██████████| 50/50 [03:19<00:00,  3.98s/it]

[I 2025-12-05 18:36:53,142] Trial 49 finished with value: 0.7375 and parameters: {'n_estimators': 430, 'max_depth': 9, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 17 with value: 0.7875.


In [140]:
print(study.best_params)
print(study.best_value)

{'n_estimators': 450, 'max_depth': 5, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 'log2'}
0.7875


In [141]:
updated_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(**study.best_params, random_state=42))
])

In [142]:
updated_pipeline.fit(X_train, y_train)
y_pred = updated_pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.65

In [143]:
import pickle 
pickle_model_path = 'insurance_model.pkl'
with open(pickle_model_path, 'wb') as f:
    pickle.dump(pipeline, f)

In [147]:
df_feat.head()

,age,income_lpa,BMI,smoker,occupation,Age Group,City Tier,Lifestyle Risk,insurance_premium_category
0,67,2.92,49.227482,False,retired,Senior Citizen,2,Low,High
1,36,34.28,30.189017,False,freelancer,Young Adult,1,Low,Low
2,39,36.64,21.118382,False,freelancer,Young Adult,2,Low,Low
3,22,3.34,45.535900,True,student,Young Adult,1,High,Medium
4,69,3.94,24.296875,True,retired,Senior Citizen,2,Low,High
